In [ ]:
! pip install pypdf langchain_openai langchain_community faiss-cpu --quiet

In [ ]:
# set openai key

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("openai")

In [ ]:
# import and define LLM

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")
llm.invoke("what can you help me with?")

In [ ]:
# parse & load PDF

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("/cookies.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [ ]:
# load to knowledge base

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

faiss_index.save_local('cookies')


In [ ]:
 # Loading the saved embeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI

embeddings = OpenAIEmbeddings()

loaded_vectors = FAISS.load_local("cookies", embeddings, allow_dangerous_deserialization=True)

# ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever= loaded_vectors.as_retriever()
)

chat_history = []
def rag(query):

  response = qa({"question":query, "chat_history":chat_history})
  chat_history.append((query, response['answer']))


  return response['answer'].strip()

In [ ]:
rag('How long do I beat the eggs and sugar for?')

In [ ]:
rag('I want to make all 3 recipes and compare them, make me a grocery list')

In [ ]:
rag('How long does the vegan recipe take and at what temperature?')